In [1]:
pip install julep

In [ ]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4()

print(AGENT_UUID)

In [3]:
import time
import uuid
import requests
from julep import Client
import os
from datetime import datetime

# Configuration. Sorry have to put agent id manually, didn't get the time to automate that(but it's pretty simple).
JULEP_API_KEY = os.getenv("JULEP_API_KEY", "_ACTUAL_JULEP_KEY_HERE")
AGENT_UUID = "_ACTUAL_AGENT_UUID_HERE"
OPENWEATHER_API_KEY = "OPENWEATHERMAP_KEY_HERE"

client = Client(api_key=JULEP_API_KEY)

In [4]:
#why not use the pre-built functionality of Julep? because coding it was easier, the integrating code of Julep isn't as seamless. I think ease of use is what we're looking for.
#Additionally I wanted to try different things with Julep than this.
#Hope this doesn't get me disqualified.
def get_current_weather(city):
    """Get current weather for a city using OpenWeatherMap API"""
    try:
        response = requests.get(
            "http://api.openweathermap.org/data/2.5/weather",
            params={
                "q": city,
                "appid": OPENWEATHER_API_KEY,
                "units": "metric"
            },
            timeout=10
        )

        if response.status_code == 200:
            data = response.json()
            temp = data['main']['temp']
            feels_like = data['main']['feels_like']
            description = data['weather'][0]['description'].title()
            humidity = data['main']['humidity']

            weather_info = f"{city} Weather: {temp}°C (feels like {feels_like}°C), {description}, Humidity: {humidity}%"

            if temp >= 25:
                weather_info += " - Perfect for outdoor dining!"
            elif temp <= 10:
                weather_info += " - Great for warm comfort foods!"
            else:
                weather_info += " - Pleasant weather for any dining experience!"

            return weather_info
        else:
            return f"Weather info for {city}: Please check local weather"

    except Exception as e:
        return f"Weather info for {city}: Unable to fetch current data"


In [5]:
#Actual Workflow
def create_foodie_workflow():
    """Create the foodie tour workflow with weather integration"""
    return {
        "name": "Foodie Tour with Weather",
        "description": "Comprehensive foodie tour generator with current weather considerations",
        "input_schema": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "City name"},
                "weather_info": {"type": "string", "description": "Current weather information"}
            },
            "required": ["city", "weather_info"]
        },
        "main": [
            {
                "prompt": [
                    {
                        "role": "system",
                        "content": "You are a world-class food tourism expert with extensive knowledge of global cuisines and restaurants. You consider current weather conditions when making dining recommendations. Structure your response with clear sections and formatting."
                    },
                    {
                        "role": "user",
                        "content": """Create a comprehensive one-day foodie tour for {{_.city}}.

Current weather: {{_.weather_info}}

Please structure your response with the following sections:

# BREAKFAST
- Weather-appropriate morning dishes
- Specific restaurant recommendations with addresses
- Why these places are special to {{_.city}}'s food culture

# LUNCH
- Signature local dishes
- Venue recommendations (consider indoor/outdoor based on weather)
- Cultural significance of the food

# DINNER
- Fine dining or traditional experiences
- Weather-suitable venues
- Reservation tips and timing

# SNACKS & DRINKS
- Must-try items between meals
- Weather-appropriate beverages
- Best locations and timing

# WEATHER CONSIDERATIONS
- How today's weather enhances the dining experience
- Timing recommendations based on conditions
- Indoor vs outdoor venue suggestions

# CULTURAL CONTEXT
- Brief history of {{_.city}}'s food scene
- What makes the local cuisine unique
- Connection between food and local culture

Provide specific restaurant names, addresses when possible, and explain why each recommendation showcases {{_.city}}'s culinary identity."""
                    }
                ],
                "unwrap": True
            }
        ]
    }


In [6]:
#Converting it to a tour.
def generate_foodie_tour(city):
    """Generate a foodie tour for a specific city"""
    print(f"\nCreating foodie tour for {city}")
    print("-" * 50)

    # Get current weather
    print("Fetching weather information...")
    weather_info = get_current_weather(city)
    print(f"   {weather_info}")

    # Create workflow
    workflow = create_foodie_workflow()
    task_id = str(uuid.uuid4())

    try:
        # Create task
        print("Setting up tour generation...")
        task = client.tasks.create_or_update(
            task_id=task_id,
            agent_id=AGENT_UUID,
            **workflow
        )

        # Execute with weather information
        print("Generating personalized tour...")
        execution = client.executions.create(
            task_id=task_id,
            input={
                "city": city,
                "weather_info": weather_info
            }
        )

        # Monitor execution with progress indicator
        start_time = time.time()
        print("Processing", end="")

        for i in range(30):  # 5 minutes max
            execution = client.executions.get(execution.id)
            elapsed = int(time.time() - start_time)

            if execution.status == 'succeeded':
                print(f"\nTour generated successfully! ({elapsed}s)")
                return True, execution.output, weather_info
            elif execution.status == 'failed':
                print(f"\nGeneration failed ({elapsed}s)")
                error_msg = getattr(execution, 'output', 'Unknown error')
                return False, error_msg, weather_info
            elif execution.status in ['cancelled']:
                print(f"\nGeneration cancelled ({elapsed}s)")
                return False, "Cancelled", weather_info

            # Progress dots
            if i % 3 == 0:
                print(".", end="", flush=True)

            time.sleep(10)

        print(f"\nTimeout after 5 minutes")
        return False, "Timeout", weather_info

    except Exception as e:
        print(f"\nError: {e}")
        return False, str(e), weather_info


In [7]:
def display_tour(city, tour_content, weather_info=None):
    """Display the tour in a structured format"""
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M")

    print(f"\n{'='*80}")
    print(f"FOODIE TOUR FOR {city.upper()}")
    print(f"Generated: {current_time}")
    if weather_info:
        print(f"{weather_info}")
    print(f"{'='*80}")
    print()
    print(tour_content)
    print()
    print(f"{'='*80}")


In [8]:
#Had a good tough time with getting the code to run, so had to seperately check if everthing is in place.
def test_connection():
    """Test Julep connection"""
    try:
        agent = client.agents.get(AGENT_UUID)
        print(f"Connected to Julep - Agent: {agent.name}")
        return True
    except Exception as e:
        print(f"Julep connection failed: {e}")
        return False


In [9]:
def main():
    """Main function - generate tours for multiple cities"""
    print("🚀 FOODIE TOUR GENERATOR WITH WEATHER INTEGRATION")
    print("="*60)

    # Testing connection
    if not test_connection():
        print("Cannot proceed without Julep connection")
        return

    # Cities to generate tours for
    cities = ["Paris", "Mumbai"]

    print(f"\nGenerating weather-aware tours for {len(cities)} cities...")

    successful_tours = []
    failed_cities = []

    for i, city in enumerate(cities, 1):
        print(f"\n[{i}/{len(cities)}] Processing {city}...")

        try:
            success, result, weather = generate_foodie_tour(city)

            if success:
                successful_tours.append((city, result, weather))
                display_tour(city, result, weather)

                if i < len(cities):  # Don't pause after last city
                    print("\n⏳ Moving to next city in 3 seconds...")
                    time.sleep(3)
            else:
                failed_cities.append((city, result))
                print(f"❌ Failed to generate tour for {city}: {result}")

        except Exception as e:
            failed_cities.append((city, str(e)))
            print(f"❌ Error with {city}: {e}")

    # Final summary
    print(f"\nGENERATION COMPLETE")
    print("-" * 30)
    print(f"Successful tours: {len(successful_tours)}")
    print(f"Failed tours: {len(failed_cities)}")

    if successful_tours:
        print(f"\n🎉 Successfully generated tours for:")
        for city, _, _ in successful_tours:
            print(f"   • {city}")

    if failed_cities:
        print(f"\nFailed to generate tours for:")
        for city, error in failed_cities:
            print(f"   • {city}: {error}")


In [10]:
#started with a single city, and then took it up to multiple.
def single_city_tour(city_name):
    """Generate tour for a single city"""
    print("SINGLE CITY FOODIE TOUR GENERATOR")
    print("="*40)

    if not test_connection():
        print("Cannot proceed without Julep connection")
        return None

    success, result, weather = generate_foodie_tour(city_name)

    if success:
        display_tour(city_name, result, weather)
        return result
    else:
        print(f"Failed to generate tour for {city_name}: {result}")
        return None


In [11]:
if __name__ == "__main__":
    # Choose one of the following options:

    # Option 1: Generate tours for multiple cities
    main()

    # Option 2: Generate tour for single city (uncomment and modify)
    # single_city_tour("Barcelona")

    # Option 3: Interactive mode (uncomment)
    # city = input("Enter city name: ").strip()
    # if city:
    #     single_city_tour(city)

🚀 FOODIE TOUR GENERATOR WITH WEATHER INTEGRATION
Connected to Julep - Agent: Foodie Tour Agent

Generating weather-aware tours for 2 cities...

[1/2] Processing Paris...

Creating foodie tour for Paris
--------------------------------------------------
Fetching weather information...
   Paris Weather: 18.79°C (feels like 18.06°C), Broken Clouds, Humidity: 51% - Pleasant weather for any dining experience!
Setting up tour generation...
Generating personalized tour...
Processing.
Tour generated successfully! (20s)

FOODIE TOUR FOR PARIS
Generated: 2025-06-08 12:04
Paris Weather: 18.79°C (feels like 18.06°C), Broken Clouds, Humidity: 51% - Pleasant weather for any dining experience!

Certainly! Here's a comprehensive foodie tour for Paris perfectly tailored to the pleasant weather conditions you provided.

---

# BREAKFAST

## Morning Dishes
### Weather-appropriate Selections:
- **Croissants and pain au chocolat**: Ideal for a light, buttery start.
- **Café crème**: A rich coffee that comp

In [13]:
#Closing Remarks:
#Had a funtime, absolutely loved the concept of put everything you need in a yaml file we'll take care of the rest.
#But it's definitely not intuitive for beginners, I myself have worked on AI projects so I kinda had an idea where to run to and so on.
#Documentation honestly felt a little incomplete, agent ids management I think I need to dig deep into on my end.
#And a good problem to have but integrations are limited.
#But having used a lot of other, close to LCNC type platforms, this one felt more robust, flexible cause most LCNC takes out flexibility of components they build. No point in having a ready-made component if I can't change the output formats and so on.
#Thank you for the opportunity.